<a href="https://www.kaggle.com/code/maryamkamelan/marketing-ab-test?scriptVersionId=204381144" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import chi2_contingency

Reading the dataset

In [ ]:
df=pd.read_csv('/kaggle/input/marketing-ab-test/marketing_AB.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe(include=["int"])

In [ ]:
df.describe(include="object")

Data Cleaning

In [ ]:
# Removing the column with no name(if there is)
if "Unnamed: 0" in df.columns:
   
    df = df.drop(["Unnamed: 0"], axis =1)

In [ ]:
df.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)
df.head(1)

In [ ]:
#checking for duplicate rows in the DataFrame 
dups = df.duplicated()
print(dups.any())

In [ ]:
df[df["user_id"].duplicated()].count()

In [ ]:
print(f'Rows            : {df.shape[0]}')
print(f'Columns         : {df.shape[1]}')
print(f'Features        : {df.columns.tolist()}')
print(f'Missing Values  : {df.isnull().values.sum()}')
print(f'Unique Values   : \n\n{df.nunique()}')

In [ ]:
#Converting the boolean datatype to integer 
df["converted_int"] = df['converted'].apply(lambda x:1 if x== True  else  0)
df["converted_int"].sum()

In [ ]:
df.head()

EDA

In [ ]:
def val_count(column):
    plt.figure(figsize=(7,5))
    sns.countplot(data=df, x=column, palette='Set2')
    plt.title(f'Value Count - {column}')
    plt.show()

In [ ]:
#loop through columns
for i in df.columns[[1,2,4,5]]:
    val_count(i)

Most of the users saw the ads compared to those saw the psa.

The most ads seen by a user occurred on Friday and then Monday.

Between 10AM and 3PM, users saw the most ads.

In [ ]:
Chart =df.groupby(by =['most_ads_day', 'test_group']).sum()['converted_int'].unstack('test_group').plot(kind= 'bar', figsize= (4,3), grid= True, stacked= True)
Chart.set_ylabel('converted_int')
Chart.set_title('Most ad days and converted_int')
plt.show()

Visualizing the total converted_int values for different combinations of most_ads_day and test_group using a stacked bar chart. It seems that on Monday we have the highest number of affected audiences.

In [ ]:
grouped_counts = df.groupby('test_group')['converted'].value_counts()

# Plot the data as a pie chart
grouped_counts.plot.pie(figsize= (4,4),autopct='%1.1f%%')
plt.title('Distribution of Converted by Test Group')
plt.ylabel('')

plt.show()

In both test groups the majority did not get affected by the ad or psa.

In [ ]:
# Group the DataFrame by 'most_ads_day' and calculate the mean conversion rate
df_day_conv = df.groupby('most_ads_day')['converted'].mean().reset_index()

# Plot the conversion rate by day using a bar chart with custom color
plt.figure(figsize=(7, 3))
plt.bar(df_day_conv['most_ads_day'], df_day_conv['converted'], color='lightgreen')  
plt.title('Conversion Rate by Day')
plt.xlabel('Day')
plt.ylabel('Conversion Rate')
plt.axhline(df['converted'].mean(), color='g', linestyle='--', label='Average')
plt.legend()
plt.grid(True)
plt.show()

Chi2 Test

In [ ]:
# Contingency table (cross-tabulation)
contingency_table = pd.crosstab(df['test_group'], df['converted'],margins=True)
contingency_table

In [ ]:
# Chi-square test
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

In [ ]:
if p_value < 0.05:
    print ( 'There is a significant difference between the two groups.')
else:
    print ('There is no significant difference between the two groups.')

In [ ]:
# Print the results
print("Chi-square value:", chi2)
print("P-value:", p_value)
print("Degrees of freedom:", dof)
print("Expected frequencies table:")
print(expected)

It suggests that there is sufficient evidence to reject the null hypothesis. Statically small p-value indicates that the observed association between the categorical variables (in this case, 'test_group' and 'converted') in the contingency table is unlikely to have occurred by chance alone. Therefore, we would conclude that there is a significant relationship between the two variables.
In this case, we can say that viewing ad had an impact on the conversion rate of users.